#과일 이름 맞추기
크기, 모양, 무게, 가격, 색상, 맛의 관련 데이터로부터 과일의 이름을 맞추는, 다중 분류 문제를 해결해 봅시다.

본 과제는 2점 만점이며, 정확도(Accuracy)를 평가 지표로 사용합니다.
Baseline 즉, 1.0의 정확도 달성: 2.0점
Baseline 즉, 1.0보다 낮은 정확도 달성: 1.0점
미제출: 0점
(주의) 허용할 수 없을 정도로 낮은 Accuracy로 기록된 것도 미제출로 간주될 수 있습니다.

현재 Baseline은 Keras 라이브러리의 Dense 클래스로 구성된 인공신경망 모델을 활용해 도출됐습니다.
데이터 세트 분할 시, random_state 값은 42로 지정했습니다.
수업 시간에 다뤘던 인공신경망 모델 구조 설계 및 하이퍼파라미터와 관련된 내용을 복습하고, 과제 2를 해결해 봅시다.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# 1. 데이터 로드
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("submission.csv")

X = train.drop(columns=["fruit_name"])
y = train["fruit_name"]

# 2. Label Encoding (문자 라벨 → 숫자)
fruit_labels = sorted(y.unique())
label_to_index = {label: i for i, label in enumerate(fruit_labels)}
index_to_label = {i: label for label, i in label_to_index.items()}

y_index = y.map(label_to_index)
y_onehot = to_categorical(y_index)

# 3. 문자형 + 숫자형 컬럼 구분
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

# 4. 전처리 파이프라인
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols)
    ]
)

# 학습용 데이터 전처리
X_processed = preprocess.fit_transform(X)
X_test_processed = preprocess.transform(test)

# 5. Keras 모델 구성 (기본 MLP)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_processed.shape[1],)),
    Dense(64, activation='relu'),
    Dense(len(fruit_labels), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 6. Train/Validation split
X_train_p, X_val_p, y_train_p, y_val_p = train_test_split(
    X_processed, y_onehot, test_size=0.2, random_state=42
)

# 7. 모델 훈련
model.fit(
    X_train_p, y_train_p,
    validation_data=(X_val_p, y_val_p),
    epochs=50,
    batch_size=8,
    verbose=0
)

# 8. test 예측
test_pred_probs = model.predict(X_test_processed)
test_pred_idx = np.argmax(test_pred_probs, axis=1)

y_pred_fruit = [index_to_label[i] for i in test_pred_idx]


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [3]:
df = pd.read_csv("submission.csv")
df.dropna(axis=1, inplace=True)

# 제출용 예측값 입력
df["fruit_name"] = y_pred_fruit
df.to_csv("new_submission.csv", index=False)

print("🎉 new_submission.csv 저장 완료!")


🎉 new_submission.csv 저장 완료!
